In [1]:
"""This code will crate a master table for each tissue"""
#Set up
COLORS = {
    "Artery-Aorta":"salmon",
    "Artery-Tibial": "red",
    "Adipose-Subcutaneous": "darkorange",    
    "Adipose-Visceral":"orange",
    "Brain-Caudate":"lemonchiffon"   , 
    "Brain-Cerebellum":"yellow",
    "Cells-Transformedfibroblasts": "skyblue",
    "Esophagus-Mucosa": "sienna",
    "Esophagus-Muscularis":"burlywood",
    "Heart-LeftVentricle":"darkviolet",
    "Lung": "greenyellow",
    "Muscle-Skeletal": "mediumslateblue",
    "Nerve-Tibial":"gold",
    "Skin-NotSunExposed":"blue",
    "Skin-SunExposed":"cornflowerblue",
    "Thyroid":"green",
    "WholeBlood": "m"
        }

SHORTEN = {
    "Artery-Aorta":"Artery A."     ,
    "Artery-Tibial": "Artery T.",
    "Adipose-Subcutaneous": "Adipose S.",    
    "Adipose-Visceral":"Adipose V.",
    "Brain-Caudate":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "E. Mucosa",
    "Esophagus-Muscularis":"E Muscularis",
    "Heart-LeftVentricle":"Ventricule",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve",
    "Skin-NotSunExposed": "Skin Unexposed",
    "Skin-SunExposed":"Skin Leg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood"
}

import math
import numpy as np
import os
import pandas as pd
import random
import sys
from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

tissu = sorted(COLORS.keys())
CHR = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]
Gene_table = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/FEATURES/Genes_only_table', sep='\t')
Motifs = pd.read_csv('/storage/resources/dbase/human/hg19/hg19.hipstr_reference_withmotif.bed', sep='\t', header=None)
Motifs.columns = ['chrom','str.start','str.end','motif.len','motif']
path='/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'

In [2]:
A=[]
B=[]
All_eSTRs=[]
for Tissue in tissu[0:]: 
    #command='mv %s%s/Master.table %s%s/oldMaster.table'%(path,Tissue,path,Tissue)
    #os.system(command)
    CAUSE=path+Tissue+'/HH/CAVIAR_wg.csv'
    SCORE='%s%s/HH/intermediate_files_caviar/strs_scorechr'%(path,Tissue)
    LR = '%s%s/PQValues'%(path,Tissue)
    ANOV = '%s%s/HH/ANOVA_pq_wg.csv'%(path,Tissue)
#LR
    lr = pd.read_csv(LR,sep='\t')
    print('lr in ',Tissue, lr.shape)
#Anova
    anova = pd.read_csv(ANOV, sep='\t')
    anova = anova[['chrom','gene','str.start','anova_qval','estr_fdr','esnp_fdr','delta_bic','delta_aic','anova_pval']]
    print('anova',anova.loc[anova['anova_qval']<0.05].shape)
    #merge 1
    lr = pd.merge(lr, anova, on=['chrom','gene','str.start'], how='left')
    print(lr.shape)
#Causal 
    causal = pd.read_csv(CAUSE, sep='\t')
    causal['chrom']=causal['CHROM']
#Scores
    score_items=[]
    for i in CHR:
        C=pd.read_csv(SCORE+str(i), sep='\t', header=None)
        score_items.append(C)
    scores=score_items[0]
    for i in range(1, len(score_items)):
        scores=scores.append(score_items[i]) 
    scores.columns=['chrom', 'gene', 'str.id', 'caviar.score']
    CA = scores; CA.index = CA['gene']        
#Merge all in table
    Table = pd.merge(lr, CA, on=['chrom','gene','str.id'], how='left')
    Table.index = Table['gene']
    Master = pd.merge(Table,causal, on=['chrom','gene'], how='left')
    print('With Caviar: ',CA.shape,Master.shape)
#Gene names
    genes=Gene_table.loc[Gene_table['gene'].isin(list(Master['gene']))]
    genes=genes[['gene','chrom','gene.name']] 
    Master=pd.merge(Master, genes, on=['chrom','gene'])
#motifs
    Master=pd.merge(Master, Motifs [['chrom','str.start','motif','str.end']], on=['chrom','str.start'])
#Additional columns
    Master['best.str'] = np.where(Master['qvalue']<=1, 1,0)
    Master['top.str.rank']=Master['str.rank']
    Master['top.snp.score']=Master['top_snp_score']
    Master['significant']= np.where(Master['qvalue']<=0.1, 1,0)
    #Master['llsignificant'] = Master['llsignif'].astype(int)
    Master['top_str'] = Master['top_str'].astype(str)
    Master['top.variant'] = np.where(Master['top.str.score']>Master['top.snp.score'], Master['top_str'], Master['top_snp'])
    Master['best.str.start'] = Master['top_str'].astype(str).apply(lambda x: int(x.split('_')[1]) if 'STR' in x else '' )
#Summarize and save
    #Head=[['gene','chrom','str.start','gene.name','str.id','best.str','best.str.start', 'best.str.score','causality.score','top.variant','top.variant.score','caviar.score', 'significant','anova_pval','delta_bic','delta_aic','p.wald', 'beta','qvalue','llqvalue','NTEST','beta.se','num.strs.in.top5','estr_fdr','esnp_fdr','motif','top_snp','str.end',]]
    Header = ['gene','chrom','str.start','str.id','best.str','top.variant','beta','qvalue','llqvalue','p.wald','NTEST','top_str','delta_bic','delta_aic','anova_pval', 'anova_qval','estr_fdr','esnp_fdr','caviar.score','num.strs.in.top5', 'top.str.score','top_snp','best.str.start', 'top.snp.score','top.str.rank','significant','beta.se','motif','gene.name','str.end']
    Master = Master[Header].sort_values(['chrom','gene','str.start'])
    Master.to_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/Master.table', sep='\t',  index=None)
    print('With gene name and motif : ',Master.shape)
    #break
##   RUN The following
##   python ~/projects/GTEX_eSTRs/gtex-estrs/Scripts/STRs-vs-SNPs_effects/merge_causality_scores.py /storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/ WholeBlood,Cells-Transformedfibroblasts,Muscle-Skeletal,Lung,Artery-Tibial,Adipose-Subcutaneous,Esophagus-Mucosa causality > Merged_Best_causality.Table
##   To merge N select best by tissue

lr in  Adipose-Subcutaneous (281318, 11)
anova (1097, 9)
(281322, 17)
With Caviar:  (62919, 4) (281322, 25)
With gene name and motif :  (281322, 30)
lr in  Adipose-Visceral (281223, 11)
anova (384, 9)
(281226, 17)
With Caviar:  (31614, 4) (281226, 25)
With gene name and motif :  (281226, 30)
lr in  Artery-Aorta (277044, 11)
anova (596, 9)
(277045, 17)
With Caviar:  (42054, 4) (277045, 25)
With gene name and motif :  (277045, 30)
lr in  Artery-Tibial (275588, 11)
anova (1054, 9)
(275594, 17)
With Caviar:  (60950, 4) (275594, 25)
With gene name and motif :  (275594, 30)
lr in  Brain-Caudate (278997, 11)
anova (99, 9)
(278997, 17)
With Caviar:  (11198, 4) (279200, 25)
With gene name and motif :  (279200, 30)
lr in  Brain-Cerebellum (277449, 11)
anova (233, 9)
(277450, 17)
With Caviar:  (25946, 4) (277450, 25)
With gene name and motif :  (277450, 30)
lr in  Cells-Transformedfibroblasts (267909, 11)
anova (914, 9)
(267911, 17)
With Caviar:  (54827, 4) (267911, 25)
With gene name and motif :

In [ ]:
F=pd.read_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/WholeBlood/Master.table', sep='\t')
sum(F['significant'])

In [ ]:
E = Master.loc[Master['gene']=='ENSG00000000457.9'].copy()
E['NTEST'] = E['NTEST'].fillna(0)
E['best.str.start']=[E.loc[E['best.str']==1]['str.start'].values[0]]*E.shape[0]
E[['top.str.score', 'top_snp', 'best.str.start', 'top.snp.score','top.str.rank', 'significant', 'beta.se', 'motif', 'gene.name','str.end']]